In [1]:
%pip install -r requiremnets.txt

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install einops

## Path

In [2]:
# Set paths in your notebook
model_path = "E:/CSE299/chatbot/llm"
embedding_save_path = "E:/CSE299/chatbot/Embedding"
pdfs_path = "E:/CSE299/chatbot/docs"



## ** title Run this to terminal**

 curl -fsSL https://ollama.com/install.sh | sh

 ollama serve

 ollama pull llama3

In [4]:
import subprocess


def serve_ollama():
    """Starts Ollama server."""
    serve_command = "ollama serve"
    try:
        subprocess.run(serve_command, shell=True, check=True)
        print("Ollama server is running.")
    except subprocess.CalledProcessError as e:
        print(f"Failed to start Ollama server: {e}")
        return




In [ ]:
# install_ollama()   # Install Ollama


Ollama installed successfully.


In [ ]:
# serve_ollama()     # Start the Ollama server

In [ ]:
# pull_model()       # Pull Llama3 model

## Import Dependencies

In [1]:
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_community.embeddings.ollama import OllamaEmbeddings
from langchain_community.llms.ollama import Ollama
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough


## Chunking & PDF loading

Load the PDF

In [2]:
#@title Function to load all PDFs from a given directory using PyPDFDirectoryLoader.

def load_pdfs_from_directory(directory_path):

    loader = PyPDFDirectoryLoader(directory_path)
    docs = loader.load()
    return docs


### Recursive Chunking

In [3]:
#@title recursive Chuking of the documents

def recursive_chunks(documents, chunk_size, chunk_overlap):

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap, add_start_index=True)
    all_splits = text_splitter.split_documents(documents)

    return all_splits

### Character Chunking

In [ ]:
#@title character Chuking of the documents
from langchain_text_splitters import CharacterTextSplitter

def character_chunks(documents, chunk_size, chunk_overlap):

    # Initialize the CharacterTextSplitter with specified parameters
    splitter = CharacterTextSplitter(
        separator="\n\n",
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        is_separator_regex=False,
    )

    # Split the text into chunks for each document
    # all_chunks = []
    # for doc in documents:
    #     # Split each document's content and extend the list
    #     chunks = splitter.split_text(doc.page_content)
    #     all_chunks.extend(chunks)
    all_chunks = splitter.split_documents(documents)

    return all_chunks



### Semantic Chunking

In [ ]:
#@title semantic Chuking of the documents
from langchain_experimental.text_splitter import SemanticChunker

def semantic_chunks(documents, embed_model):

    semantic_chunker = SemanticChunker(embed_model, breakpoint_threshold_type="percentile")

    # Create semantic chunks from the documents' content
    semantic_chunks = semantic_chunker.create_documents([d.page_content for d in documents])

    return semantic_chunks


## Intializing Models

Embedding Model

In [4]:
from langchain_huggingface import HuggingFaceEmbeddings

def get_embedding_model(model_name, model_kwargs, path):
    encode_kwargs = {'normalize_embeddings': True}

    # Initialize HuggingFaceEmbeddings with model name and kwargs
    hf = HuggingFaceEmbeddings(
        model_name=model_name,
        model_kwargs=model_kwargs,
        encode_kwargs=encode_kwargs,
        cache_folder=path
    )

    return hf

LLM (Ollama model)

In [5]:
# @title get Ollama model

from langchain_community.llms import Ollama

def get_ollama_model(model):
    llm = Ollama(model=model)
    return llm


#  Embedding 

In [ ]:
# # @title Load embedding model
# from transformers import AutoModel
# embedding_model_name = "nomic-ai/nomic-embed-text-v1.5"
# model_kwargs = {'device': 'cpu', 'trust_remote_code': True}  # Pass trust_remote_code here
# embedding = get_embedding_model(embedding_model_name, model_kwargs, path="embeddings/nomic/")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/547M [00:00<?, ?B/s]

In [10]:
import os
from transformers import AutoModel


# Define paths for saving the embedding model and vector database
embedding_model_save_path = r"E:\CSE299\chatbot\llm\nomic"
embedding_vector_db_path = r"E:\CSE299\chatbot\Embedding\nomic"

# Ensure directories exist
os.makedirs(embedding_model_save_path, exist_ok=True)
os.makedirs(embedding_vector_db_path, exist_ok=True)

# Load the embedding model and specify the local save path for the model
embedding_model_name = "nomic-ai/nomic-embed-text-v1.5"
model_kwargs = {'device': 'cpu', 'trust_remote_code': True}

# Assuming get_embedding_model is defined to support a 'path' argument for saving locally
embedding = get_embedding_model(embedding_model_name, model_kwargs, path=embedding_model_save_path)

KeyboardInterrupt: 

In [ ]:
sem_embedding = embedding

Dimension

In [ ]:
# Access the underlying model from the HuggingFaceEmbeddings instance
hf_model = embedding.client  # 'client' holds the underlying Hugging Face model

# Get the embedding dimension using the dedicated method
embedding_dim = hf_model.get_sentence_embedding_dimension()

# Print the embedding dimension
print(f"Embedding Dimension: {embedding_dim}")

## Make Chunks 

In [ ]:
#@title chunking

docs = load_pdfs_from_directory(pdfs_path)

all_splits = recursive_chunks(docs, 1000, 200);
# all_splits = semantic_chunks(docs, sem_embedding);
# all_splits = character_chunks(docs, 1000, 200)

In [ ]:
#@title dbstore

#vectorstoring
vectorstore = Chroma.from_documents(documents = all_splits, embedding = embedding, persist_directory=embedding_vector_db_path)
retriever = vectorstore.as_retriever(search_type = "similarity", search_kwargs = {"k" : 6})


NameError: name 'all_splits' is not defined

# Existing Vector DB load

In [ ]:
from langchain.vectorstores import Chroma

vectorstore = Chroma(persist_directory=embedding_vector_db_path, embedding_function=embedding)

retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [ ]:
# from langchain.vectorstores import Chroma

# # Load existing vector store (you don't need to recreate it)
# vectorstore = Chroma(persist_directory='db/nomic/semantic/',embedding_function=embedding)
# retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})


Prompt

In [ ]:
#@title prompt template

PROMPT_TEMPLATE = """
Use the following context to answer the question clearly and simply:
{context}
Answer the question based on the above context: {question}.
Provide a detailed answer.
Do not justify your answers.
Please explain using simple language, relevant examples, and avoid jargon.
"""



In [ ]:
llm = get_ollama_model("llama3.2:1b")
# prompt = process_query("explain newtons first law with example")
# response = llm(prompt)
# print(response)


In [ ]:
llm.invoke("explain newtons first law with example")

'Newton\'s First Law of Motion, also known as the Law of Inertia, states that an object at rest will remain at rest, and an object in motion will continue to move with a constant velocity, unless acted upon by an external force.\n\nHere\'s a simple example to illustrate this concept:\n\n**Example: A Basketball Player on the Court**\n\nImagine a basketball player named John who is sitting on the court, watching a game. He has just taken his seat and is enjoying the game. Suddenly, the coach calls out to him, "John, come over here and try your shot!"\n\nJohn gets up from his seat and walks towards the coach\'s bench. As he approaches, he feels the urge to sit back down again. The urge is strong, but John is not moving.\n\n**Initial State:**\nJohn is at rest (not moving) on the court.\n\n**Applying a Force:**\nThe coach calls out to him, applying an external force (a push or shove). This force causes John to suddenly get up from his seat and walk towards the bench.\n\n**Result:**\nAs John

In [ ]:
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# llm = get_ollama_model("gemma2:2b")

prompt = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)

retrieval_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

for chunk in retrieval_chain.stream("explain newtons first law with example"):
    print(chunk, end="", flush=True)

Newton's First Law explains that an object will keep moving in its current direction unless something else is applied to it.

Imagine you're playing a game where you throw a ball straight up in the air. What happens? The ball comes back down to the ground and lands in your hand, right?

That's because the force you applied by throwing the ball (up) kept it moving in that same direction until something else came along and changed its motion.

Let's use an example: Imagine a car going on a straight road. If no one pushes or pulls the car, it will keep driving forward. But if someone hits the brakes suddenly, the car will start slowing down and eventually stop.

That's because the force of the brakes is acting on the car (pushing it backward), which changes its motion. The car can't just stop on its own; something has to apply a force to make it do so.

Newton's First Law applies to all objects, big or small. It explains why things keep moving in their current direction unless someone or 

Recall + Quality

In [ ]:
import time
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Embedding size - assuming you are using the 'embedding_function' from Chroma
embedding_size = embedding._embedding_fn.model.config.hidden_size  # For transformers-based models

# Step 1: Measure recall time of chunks
start_time = time.time()
retrieved_docs = retriever.get_relevant_documents("explain newtons first law with example")
end_time = time.time()

recall_time = end_time - start_time
print(f"Recall time: {recall_time:.4f} seconds")

# Step 2: Calculate embedding quality (Cosine similarity)
def calculate_embedding_quality(query_embedding, retrieved_embeddings):
    # Calculate cosine similarity between the query embedding and the retrieved embeddings
    similarities = cosine_similarity([query_embedding], retrieved_embeddings)

    # Average similarity score
    avg_similarity = np.mean(similarities)
    return avg_similarity

# Simulating query embedding and retrieved embeddings for this example
query_embedding = embedding.embed_query("explain newtons first law with example")
retrieved_embeddings = [doc.embedding for doc in retrieved_docs]

# Step 3: Calculate embedding quality
embedding_quality = calculate_embedding_quality(query_embedding, retrieved_embeddings)
print(f"Embedding Quality (Average Cosine Similarity): {embedding_quality:.4f}")
